In [21]:
import os
import json
import numpy as np

machine_labeling_dir = "/Users/eunbin/Desktop/딥러닝/Project/data/Training/라벨링데이터/기구_Labeling"
sub_dirs = os.listdir(machine_labeling_dir)[1:]

In [22]:
json_file_list = []
json_data_list = []

for sub_dir in sub_dirs:
    dir = os.path.join(machine_labeling_dir, sub_dir)
    json_files = [file for file in os.listdir(dir) if file.endswith(".json") and "3d" not in file]
    json_file_list += json_files

    for file in json_files:
        file_path = os.path.join(dir, file)
        with open(file_path, "r", encoding="utf-8") as f:
            json_data = json.load(f)
            json_data_list.append(json_data)
    print(f"load {len(json_files)} files from ({sub_dir})...")

print(f"total {len(json_data_list)} files loaded...")

load 1011 files from (Day17_201017_F)...
load 1071 files from (Day22_201023_F)...
load 1071 files from (Day20_201021_F)...
load 1071 files from (Day21_201022_F)...
total 4224 files loaded...


### Sample JSON file

In [23]:
print(f"Load first json file({json_file_list[0]})")
print(f"keys: {json_data_list[0].keys()}")

Load first json file(D17-1-816.json)
keys: dict_keys(['frames', 'type', 'type_info'])


In [24]:
print(f"frame 개수: {len(json_data_list[0]['frames'])}")
json_data_list[0]["frames"]

frame 개수: 16


[{'view1': {'pts': {'Nose': {'x': 1029, 'y': 577},
    'Left Eye': {'x': 1039, 'y': 567},
    'Right Eye': {'x': 1021, 'y': 564},
    'Left Ear': {'x': 1015, 'y': 550},
    'Right Ear': {'x': 986, 'y': 552},
    'Left Shoulder': {'x': 1021, 'y': 616},
    'Right Shoulder': {'x': 934, 'y': 603},
    'Left Elbow': {'x': 1067, 'y': 676},
    'Right Elbow': {'x': 972, 'y': 679},
    'Left Wrist': {'x': 1103, 'y': 700},
    'Right Wrist': {'x': 1055, 'y': 703},
    'Left Hip': {'x': 1008, 'y': 761},
    'Right Hip': {'x': 917, 'y': 771},
    'Left Knee': {'x': 1079, 'y': 708},
    'Right Knee': {'x': 1009, 'y': 715},
    'Left Ankle': {'x': 1110, 'y': 837},
    'Right Ankle': {'x': 1058, 'y': 859},
    'Neck': {'x': 982, 'y': 588},
    'Left Palm': {'x': 1111, 'y': 715},
    'Right Palm': {'x': 1071, 'y': 710},
    'Back': {'x': 963, 'y': 650},
    'Waist': {'x': 978, 'y': 713},
    'Left Foot': {'x': 1136, 'y': 830},
    'Right Foot': {'x': 1081, 'y': 853}},
   'active': 'Yes',
   'img_key

In [25]:
json_data_list[0]["type"]

'816'

In [26]:
json_data_list[0]["type_info"]

{'key': '816',
 'type': '기구',
 'pose': '기구',
 'exercise': '로잉머신',
 'conditions': [{'condition': '체스트 업 유지', 'value': False},
  {'condition': '상체 과도한 젖힘 없음', 'value': False},
  {'condition': '수축 시 팔 당김보다 다리 펴짐이 우선', 'value': False},
  {'condition': '이완 시 팔 펴짐이 다리 접힘보다 우선', 'value': False},
  {'condition': '어깨 으쓱 없음', 'value': False}],
 'description': '가슴 닫고 상체 눕고 팔 먼저 당기고 다리 먼저 접고 어깨 으쓱하고'}

In [27]:
json_data_list[0]["frames"][15]["view1"].keys()

dict_keys(['pts', 'active', 'img_key'])

In [28]:
len(json_data_list[62]["frames"])

16

### Preparing data

In [51]:
excercise_dict = {}
for i, excercise_name in enumerate(set([json_data_list[i]["type_info"]['exercise'] for i in range(len(json_data_list))])):
    excercise_dict[excercise_name] = i

excercise_dict

{'로잉머신': 0,
 '풀업': 1,
 '랫풀 다운': 2,
 '행잉 레그 레이즈': 3,
 '페이스 풀': 4,
 '딥스': 5,
 '케이블 푸시 다운': 6,
 '케이블 크런치': 7}

In [92]:
machine_pts_list = []
machine_label_list = []

for i in range(len(json_data_list)):
    frame_list = []
    try:
        for k, frame in enumerate([value for value in [json_data_list[i]["frames"][j]["view1"]["pts"] for j in range(16)]]):
            frame_list.append([element for sublist in [(value['x'], value['y']) for value in frame.values()] for element in sublist])
        machine_label_list.append(excercise_dict[json_data_list[i]["type_info"]['exercise']])
    except:
        continue
    machine_pts_list.append(frame_list)

In [93]:
len(machine_pts_list), len(machine_label_list)

(4215, 4215)

In [94]:
machine_pts_list = np.array(machine_pts_list)
machine_label_list = np.array(machine_label_list)

In [95]:
# 첫번째 데이터의 첫번째 frame
machine_pts_list[0][0]

array([1029,  577, 1039,  567, 1021,  564, 1015,  550,  986,  552, 1021,
        616,  934,  603, 1067,  676,  972,  679, 1103,  700, 1055,  703,
       1008,  761,  917,  771, 1079,  708, 1009,  715, 1110,  837, 1058,
        859,  982,  588, 1111,  715, 1071,  710,  963,  650,  978,  713,
       1136,  830, 1081,  853])

In [96]:
machine_lable_list[:100]

array([0, 0, 5, 0, 5, 1, 1, 4, 4, 0, 6, 0, 6, 6, 0, 6, 2, 2, 0, 7, 0, 7,
       0, 7, 0, 1, 0, 0, 1, 0, 0, 3, 3, 0, 0, 7, 6, 7, 6, 4, 2, 2, 2, 3,
       3, 5, 0, 5, 0, 1, 1, 0, 1, 0, 1, 6, 6, 0, 6, 0, 6, 2, 2, 3, 3, 5,
       5, 0, 0, 0, 0, 0, 6, 7, 7, 2, 2, 4, 4, 6, 0, 7, 6, 0, 6, 6, 2, 2,
       1, 1, 5, 5, 0, 1, 0, 1, 2, 2, 0, 7])

In [97]:
import pickle

with open('machine_pts.pkl', 'wb') as f:
    pickle.dump(machine_pts_list, f)

with open('machine_label.pkl', 'wb') as f:
    pickle.dump(machine_label_list, f)